In [378]:
%matplotlib inline

In [379]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [380]:
import plotly
plotly.tools.set_credentials_file(username='liyihan', api_key='sTd0wThycrlBnHrU9VS4')

In [381]:
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as ff
from plotly import tools

In [382]:
def self_check_cate(x):
    if x == 5:
        return 0
    if x == 4:
        return 1
    if x == 6:
        return 2
    if x == 3:
        return 3
    if x == 2:
        return 4
    elif x == 1:
        return 100000
    else:
        return x - 4

In [383]:
def find_most_common_category(data,attr):
    common_cate = []
    for n in attr:
        common_cate.append(self_check_cate(len(set(data[n]))))
    for n in range(len(common_cate)):
        if common_cate[n] == min(common_cate):
            index_n = n
    return attr[index_n]

In [384]:
def check_relative(data):
    data_rela = data.corr()
    relative_pair_temp = {}
    relative_pair = {}
    for i in range(len(data_rela)):
        for j in range(i+1,len(data_rela)):
            pair_name = data_rela.columns[i]+' and '+data_rela.columns[j]
            relative_pair_temp[pair_name] = data_rela.iloc[i,j]
    for i in relative_pair_temp:
        if abs(relative_pair_temp[i]) >= 0.001:
            relative_pair[i] = relative_pair_temp[i]  
    return relative_pair

In [392]:
def data_overview(data,fn):
    attr = [n for n in data.columns]
    cate_attr = find_most_common_category(data,attr)
    rela_dict = check_relative(data)
    file_name_temp = fn[:-4]
    file_name = file_name_temp.split('/')[-1]
    #scatter of distribution
    point_color = []
    location = []
    for i,j in zip(data['latitude'],data['longitude']):
        location.append((i,j))
    for i in location:
        point_count = 0
        for j in location:
            if i == j:
                point_count += 1
        point_color.append(point_count)
    trace_a = go.Scatter(
    x = data['latitude'],
    y = data['longitude'],
    mode='markers',
    marker=dict(
        size='10',
        color = point_color, 
        colorscale='Viridis',
        showscale=True
        )
    )
    layout_a= go.Layout(
    title= file_name+' data distribution',
    xaxis= dict(
        title= 'Latitude'
    ),
    yaxis=dict(
        title= 'Longitude'
    ))
    fig_a = go.Figure(data=[trace_a], layout=layout_a)
    
    # category pie chart
    pie_label_temp = []
    pie_value_temp = []
    pie_label = []
    pie_value = []
    cate_index = set(data[cate_attr])
    count_cate = []
    for i in cate_index:
        count_for_each = 0
        pie_label_temp.append(i)
        for j in data[cate_attr]:
            if i == j:
                count_for_each += 1
        pie_value_temp.append(count_for_each)
    pie_other = 0
    threshold = sum(pie_value_temp) * 0.10
    for i in range(len(pie_label_temp)):
        if pie_value_temp[i] >= threshold:
            pie_label.append(pie_label_temp[i])
            pie_value.append(pie_value_temp[i])
        else:
            pie_other = pie_other + pie_value_temp[i]
    if pie_other > 0:
        pie_label.append('others')
        pie_value.append(pie_other)
    trace_b=go.Pie(labels=pie_label,values=pie_value)
    layout_b= go.Layout(
    title= file_name+' '+cate_attr+' distribution',
    )
    fig_b = go.Figure(data=[trace_b], layout=layout_b)

    #bar chart of relative
    bar_label_temp = []
    bar_value_temp = []
    bar_value_temp_abs = []
    bar_label = []
    bar_value = []
    for k,v in rela_dict.items():
        bar_label_temp.append(k)
        bar_value_temp.append(v)
        bar_value_temp_abs.append(abs(v))
    for i in range(min(10,len(bar_value_temp))):
        bar_max = bar_value_temp_abs.index(max(bar_value_temp_abs))
        bar_label.append(bar_label_temp[bar_max])
        bar_value.append(bar_value_temp[bar_max])
        bar_value_temp_abs[bar_max] = 0
    trace_c = go.Bar(x = bar_label,y = bar_value)
    layout_c= go.Layout(
    title= file_name+' correlation value',
    xaxis= dict(
        title= 'variables pair'
    ),
    yaxis=dict(
        title= 'correlation value'
    ))
    fig_c = go.Figure(data=[trace_c], layout=layout_c)
    file_name_a = file_name+'_data_distribution.png'
    file_name_b = file_name+'_categroy_pie_chart.png'
    file_name_c = file_name+'_correlation_value.png'
    return py.iplot(fig_a,filename= file_name_a),py.iplot(fig_b,filename= file_name_b),py.iplot(fig_c,filename=file_name_c)
    

In [393]:
fn = "/Users/guanda/Desktop/sample_flat.csv"
fn2 ='/Users/guanda/Desktop/wgd_csv.csv'

In [394]:
dataframe_a = pd.read_csv(fn)
dataframe_b = pd.read_csv(fn2)
da,db,dc =data_overview(dataframe_a,fn)

In [395]:
da

In [396]:
db

In [397]:
dc